In [2]:
from bs4 import BeautifulSoup
import requests
import time

import pandas as pd

In [3]:
# Cделаем список моделей машин, для которых будем собирать информацию
model_all = ['BMW', 'VOLKSWAGEN', 'NISSAN', 'MERCEDES', 'TOYOTA', 'AUDI', 'MITSUBISHI', 'SKODA', 
             'VOLVO', 'HONDA', 'INFINITI', 'LEXUS']

In [4]:
# Cделаем список сайтов с объявлениями машин для последующего парсинга
pages_url_list=[]

# Для каждого бренда автомобилей пройдемся по 99-ти страницам объявлений авто.ру и вытащим страницы этих объявлений 
for brand in model_all:
    number_of_model = 0  # Счетчик для количества объявлений
    for i in range(1,99):   
        response = requests.get(f'https://auto.ru/cars/{brand}/all/?page={i}')
        page = BeautifulSoup(response.text, 'html.parser')
        for link in page.findAll('a',{"class": "Link OfferThumb"}):
            try:
                if link['href'] not in pages_url_list:  # Проверка на дубликаты
                    pages_url_list.append(link['href'])
                    number_of_model += 1
            except:
                pass
    print(f'Число машин {brand} : {number_of_model} , в совокупной сумме :{len(pages_url_list)}')   
    time.sleep(10)

Число машин BMW : 3471 , в совокупной сумме :3471
Число машин VOLKSWAGEN : 3496 , в совокупной сумме :6967
Число машин NISSAN : 3565 , в совокупной сумме :10532
Число машин MERCEDES : 3425 , в совокупной сумме :13957
Число машин TOYOTA : 3545 , в совокупной сумме :17502
Число машин AUDI : 3538 , в совокупной сумме :21040
Число машин MITSUBISHI : 3607 , в совокупной сумме :24647
Число машин SKODA : 3589 , в совокупной сумме :28236
Число машин VOLVO : 3578 , в совокупной сумме :31814
Число машин HONDA : 3643 , в совокупной сумме :35457
Число машин INFINITI : 1790 , в совокупной сумме :37247
Число машин LEXUS : 3596 , в совокупной сумме :40843


In [5]:
cars_list = []
object_number = 0
diler_avto = 0

for item in pages_url_list:
    response = requests.get(item)
    response.encoding = 'utf8'
    page = BeautifulSoup(response.text, 'html.parser')
    if page.find('div', class_='CardSold') == None:
        try:
            json_data = json.loads(
                page.find('script', type="application/ld+json").string)
        except:
            pass
        
        cls_str = 'CardInfoRow_'
        span_str = 'CardInfoRow__cell'
        try:
            cars_list.append({
            'bodyType': json_data['bodyType'],
            'brand': json_data['brand'],
            'car_url': json_data['offers']['url'],
            'color': json_data['color'],
            'description': json_data['description'],
            'engineDisplacement': json_data['vehicleEngine']['engineDisplacement'],
            'enginePower': json_data['vehicleEngine']['enginePower'],
            'fuelType': json_data['fuelType'],
            'image': json_data['image'],
            'mileage': page.find(
                'li', class_=cls_str+'kmAge').find_all('span')[1].text,
            'modelDate': json_data['modelDate'],
            'model_name': json_data['name'],
            'name': json_data['vehicleEngine']['name'],
            'numberOfDoors': json_data['numberOfDoors'],
            'parsing_unixtime': int(time.time()),
            'priceCurrency': json_data['offers']['priceCurrency'],
            'productionDate': json_data['productionDate'],
            'sell_id': page.find(
                'div', title='Идентификатор объявления').text,
            'vehicleConfiguration': json_data['vehicleConfiguration'],
            'vehicleTransmission': json_data['vehicleTransmission'],
            'Владельцы': page.find(
                'li', class_=cls_str+'ownersCount').find_all('span')[1].text,
            'ПТС': page.find(
                'li', class_=cls_str+'pts').find_all('span')[1].text,
            'Привод': page.find(
                'li', class_=cls_str+'drive').find_all('span')[1].text,
            'Руль': page.find(
                'li', class_=cls_str+'wheel').find_all('span')[1].text,
            'Состояние': page.find(
                'li', class_=cls_str+'state').find_all('span')[1].text,
            'Таможня': page.find(
                'li', class_=cls_str+'customs').find_all('span')[1].text,
            'price': page.find(
                'span', class_='OfferPriceCaption__price').text
            })
            object_number += 1
            if object_number % 500 == 0:
                print(object_number)
        except:
            diler_avto += 1
            pass
    time.sleep(0.1)
    
print("Автомобилей продаются дилерами: ", diler_avto)
print("Автомобилей охарактеризовано: ", object_number)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
Автомобилей продаются дилерами:  8419
Автомобилей охарактеризовано:  31754


In [8]:
data = pd.DataFrame(cars_list)
data.sample(5)

,bodyType,brand,car_url,color,description,engineDisplacement,enginePower,fuelType,image,mileage,...,sell_id,vehicleConfiguration,vehicleTransmission,Владельцы,ПТС,Привод,Руль,Состояние,Таможня,price
16424,внедорожник 5 дв.,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/outl...,белый,3 Владельца\nПомощь в продаже Вашего автомобил...,2.0 LTR,146 N12,бензин,https://autoru.naydex.net/9tCfb3L22/5c9f5d8C/n...,169 000 км,...,№ 1103936772,ALLROAD_5_DOORS VARIATOR 2.0,вариатор,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,965 000 ₽
22576,седан,VOLVO,https://auto.ru/cars/used/sale/volvo/460/11021...,красный,Пеменены все расходники.автомобиль в отличном ...,1.7 LTR,102 N12,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,190 000 км,...,№ 1102119334,SEDAN MECHANICAL 1.7,механическая,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,135 000 ₽
19735,хэтчбек 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/fabia/110...,красный,Продается авто.В нормальном рабочем состоянии...,1.2 LTR,64 N12,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,230 000 км,...,№ 1103877640,HATCHBACK_5_DOORS MECHANICAL 1.2,механическая,3 или более,Оригинал,передний,Левый,Не требует ремонта,Растаможен,135 000 ₽
1460,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x6/11041393...,красный,Ваш выбор и Ваше время — наши главные приорите...,3.0 LTR,249 N12,дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,161 000 км,...,№ 1104139378,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,2 849 000 ₽
9122,внедорожник 5 дв.,MERCEDES,https://auto.ru/cars/used/sale/mercedes/glk_kl...,чёрный,"ID: 47364\n\nПТС оригинал, год выпуска по ПТС ...",3.0 LTR,224 N12,дизель,https://autoru.naydex.net/9tCfb3L22/5c9f5d8C/n...,196 874 км,...,№ 1103997776,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,2 владельца,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1 039 000 ₽


In [7]:
data.to_csv('auto_ru.csv')